In [1]:
#!pip install ..
!pip install emoji
import emoji

!pip list
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

!pip install seaborn
!pip install scikit-learn
!pip install optuna

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Package                   Version
------------------------- ----------------
ace_tools                 0.0
alembic                   1.15.2
anyio                     4.8.0
apturl                    0.5.2
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
bcrypt                    3.2.0
beautifulsoup4            4.13.3
bleach                    6.2.0
blinker                   1.4
Brlapi                    0.8.3
certifi                   2020.6.20
cffi                      1.17.1
chardet                   4.0.0
charset-normalizer        3.4.1
click                     8.0.3
colorama                  0.4.4
colorlog                  6.9.0
comm                      0.2.2
command-not-found         0.3
contourpy                 1.3.0
cryptogr

In [2]:
df=pd.read_csv('scitweets_balanced.tsv', sep='\t')
display (df.head())

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0.0,3.166700e+17,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1.0,3.190909e+17,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2.0,3.220309e+17,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3.0,3.226948e+17,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4.0,3.285244e+17,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


In [4]:
!wget https://www.lirmm.fr/~poncelet/Ressources/StopWordsFrench.csv

--2025-04-15 14:02:53--  https://www.lirmm.fr/~poncelet/Ressources/StopWordsFrench.csv
Résolution de www.lirmm.fr (www.lirmm.fr)… 193.49.104.251
Connexion à www.lirmm.fr (www.lirmm.fr)|193.49.104.251|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1925 (1,9K) [text/csv]
Enregistre : ‘StopWordsFrench.csv’

StopWordsFrench.csv 100%[===================>]   1,88K  --.-KB/s    ds 0s      

2025-04-15 14:02:53 (924 MB/s) - ‘StopWordsFrench.csv’ enregistré [1925/1925]



In [5]:
# =============================================================================
# 1. Chargement des données et préparation
# =============================================================================
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le fichier TSV contenant vos données
df = pd.read_csv('scitweets_balanced.tsv', sep='\t')
# Afficher un extrait pour vérification
display(df.head())

# S'assurer qu'il n'y a pas de valeurs manquantes dans la colonne 'text'
df.dropna(subset=['text'], inplace=True)

# =============================================================================
# 2. Chargement de la liste des stopwords français
# =============================================================================
# Le fichier StopWordsFrench.csv doit être dans votre répertoire courant.
# Ici, on suppose que le fichier est organisé avec les stop words en index.
stopwords_df = pd.read_csv("StopWordsFrench.csv", sep=',', index_col=0)
# Conversion en liste. Selon la structure, ici on récupère les index comme stopwords.
french_stopwords = stopwords_df.index.tolist()

# Optionnel : vérifier les stop words chargés
print("Stopwords français chargés (extrait) :", french_stopwords[:10])

# =============================================================================
# 3. Création du TfidfVectorizer avec les paramètres souhaités
# =============================================================================
# Nous configurons le vectorizer pour :
# - Ne pas convertir les textes en minuscules (lowercase=False)
# - Utiliser notre liste personnalisée de stop words français
# - Générer des n-grammes de taille 1 et 2 (ngram_range=(1,2))
# - Limiter le nombre total de features à 15 (max_features=15)
vectorizer = TfidfVectorizer(
    lowercase=False,
    stop_words=french_stopwords,
    ngram_range=(1,2),
    max_features=15
)

# Appliquer le fit_transform sur la colonne 'text' de votre DataFrame
X_tfidf = vectorizer.fit_transform(df['text'])

# =============================================================================
# 4. Visualisation et vérification de la matrice TF-IDF
# =============================================================================
# Création d'un DataFrame pour afficher les valeurs
tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print("Taille de la matrice TF-IDF :", tfidf_df.shape)
display(tfidf_df)

# Afficher la liste des termes (features) sélectionnés
print("Liste des features sélectionnées :", vectorizer.get_feature_names_out())

# Afficher l'idf de chaque terme du vocabulaire
idf_values = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))
print("Valeurs IDF de chaque terme :")
print(idf_values)


,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0.0,3.166700e+17,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1.0,3.190909e+17,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2.0,3.220309e+17,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3.0,3.226948e+17,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4.0,3.285244e+17,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


Stopwords français chargés (extrait) : []
Taille de la matrice TF-IDF : (1530, 15)


,and,are,co,for,http,http co,https,https co,in,is,of,on,that,the,to
0,0.0,0.413636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359338,0.000000,0.000000,0.836529,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.735858,0.000000,0.000000,0.677136,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.654132,0.000000,0.000000,0.000000,0.000000,0.541943,0.527645
4,0.0,0.000000,0.322776,0.000000,0.669259,0.669259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,0.0,0.000000,0.359561,0.000000,0.000000,0.000000,0.423936,0.423936,0.000000,0.715034,0.000000,0.000000,0.000000,0.000000,0.000000
1526,0.0,0.000000,0.514327,0.000000,0.000000,0.000000,0.606411,0.606411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1527,0.0,0.234402,0.000000,0.196633,0.000000,0.000000,0.000000,0.000000,0.000000,0.203632,0.487128,0.694368,0.237025,0.298836,0.000000
1528,0.0,0.000000,0.000000,0.617824,0.000000,0.000000,0.000000,0.000000,0.000000,0.639815,0.000000,0.000000,0.000000,0.000000,0.457088


Liste des features sélectionnées : ['and' 'are' 'co' 'for' 'http' 'http co' 'https' 'https co' 'in' 'is' 'of'
 'on' 'that' 'the' 'to']
Valeurs IDF de chaque terme :
{'and': np.float64(2.362414555867221), 'are': np.float64(3.1920128391550238), 'co': np.float64(1.3944224496161757), 'for': np.float64(2.6776845848378312), 'http': np.float64(2.89125868513589), 'http co': np.float64(2.89125868513589), 'https': np.float64(1.6440771264787175), 'https co': np.float64(1.6440771264787175), 'in': np.float64(2.4559406138780444), 'is': np.float64(2.772994764642156), 'of': np.float64(2.2111835861432976), 'on': np.float64(3.1518928453655985), 'that': np.float64(3.2277309217571033), 'the': np.float64(2.034727148801741), 'to': np.float64(1.9810469993381172)}
